In [ ]:
import os
from dotenv import load_dotenv, find_dotenv, dotenv_values
from openpyxl.drawing.image import Image 
from openpyxl import load_workbook, Workbook
import re
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import tempfile
from tqdm import tqdm
import time
import calendar
import datetime
from openpyxl.styles import Font, Fill, Alignment, Border, Side, PatternFill
from openpyxl.worksheet.table import Table, TableStyleInfo
from copy import copy
import sys
from urllib.parse import quote
import getpass
import bcrypt
from PIL import Image
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
configpath = find_dotenv()
config = dotenv_values(configpath)
Ventryfile = config.get('entryfile')
verifybutton = config.get('verifybutton')
country = config.get('country')
VATinput = config.get('VATinput')
VIESURL=config.get('VIESURL')
GBURL=config.get('GBURL')
valid_prefixes=config.get('valid_prefixes')
Vhalfpointpath=config.get('halfpointpath')
Vfinalpath=config.get('finalpath')
Centryfile=config.get('Centryfile')
Chalfpointpath=config.get('Chalfpointpath')
Cfinalpath=config.get('Cfinalpath')

In [ ]:
max_retries = 10
retry_count = 0

In [ ]:
wb = load_workbook(Centryfile)
ws = wb['Sheet1']

VATs = []

for row in ws.iter_rows(min_row=2, values_only=False):
	SAPVATs = row[7].value
	VATs.append(SAPVATs)

print(VATs)

In [ ]:
VIESVIABLE = valid_prefixes

print(VIESVIABLE)
options = webdriver.ChromeOptions()
options.headless = True

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(2)

actions = ActionChains(driver)

wb1 = load_workbook(Chalfpointpath)
worksheet = wb1['Sheet1']
row_counter = 2
worksheet['A1'] = 'VAT'
worksheet['B1'] = 'VIES NAME'
worksheet['C1'] = 'VIES ADDRESS'
worksheet['D1'] = 'VIES VAT STATUS'
wait = WebDriverWait(driver, 20)

for VAT in VIESVIABLE:
    if VAT is None:
        print("VAT not in SAP")
        continue

    driver.get(VIESURL)
    countryfromVAT = VAT[:2]
    countrretry = 5
    for _ in range(countrretry):
        try:
            country_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, country)))
            country_selector = Select(driver.find_element(By.CSS_SELECTOR, country))
            country_selector.select_by_value(countryfromVAT)
            break 
        except NoSuchElementException:
            print("Option 'ES' not found in dropdown. Retrying...")
    else:
        print(f"Failed to find 'ES' after {countrretry} attempts.")

    VATinput_element = driver.find_element(By.CSS_SELECTOR, VATinput)
    VATtest = VAT[2:] 
    VATinput_element.send_keys(VATtest)

    time.sleep(2)

    while retry_count <= max_retries:
        try:
            verifybutton_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, verifybutton)))
            actions.send_keys(Keys.ENTER)
            actions.perform()
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            time.sleep(1.5)
            print("Button not found. Retrying...")
            retry_count += 1

    if retry_count < max_retries:
        print("Button clicked successfully")
    else:
        print("Button not found after maximum retries")

    while retry_count <= max_retries:
        try:
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
            table = (EC.presence_of_element_located((By.TAG_NAME, 'table')))
            break
        except Exception as e:
            print(f'Error occured: {e}')
            time.sleep(1.5)
            print('No table, retrying...')
            retry_count +=1

    page_source = driver.page_source

    soup = BeautifulSoup(page_source, 'html.parser')

    text_result_element = soup.find('span', {'data-testid': 'textResult'})

    table = soup.find('table', class_='no-mobile-header ecl-table')

    text_result_element = soup.find('span', {'data-testid': 'textResult'})

    if text_result_element is not None:
        text_result = text_result_element.text
    else:
        text_result = "Element not found"

    nameandaddress = []

    if table:
        for row in table.find_all('tr'):
            cols = [ele.text.strip() for ele in row.find_all('td')]
            nameandaddress.append([ele for ele in cols if ele])
    else:
        print("Table not found on the page")

    print(text_result)

    name = None
    address = None

    for sublist in nameandaddress:
        print(sublist)
        if len(sublist) >= 2:
            if sublist[0] == 'Name':
                name = sublist[1]
            elif sublist[0] == 'Address':
                address = sublist[1]

    if name is not None and address is not None:
        print("VAT Number:", VAT)
        print("Name:", name)
        print("Address:", address)
        worksheet[f'A{row_counter}'] = VAT
        worksheet[f'B{row_counter}'] = name
        worksheet[f'C{row_counter}'] = address
        worksheet[f'D{row_counter}'] = text_result

    else:
        print(f"VAT Number {VAT} - IF ES - NOT FOUND, OTHERWISE INCORRECT VAT")
        print(text_result)
        worksheet[f'D{row_counter}'] = text_result
        worksheet[f'A{row_counter}'] = VAT


        time.sleep(2)
    row_counter += 1

    wb1.save(Chalfpointpath)
print(VIESVIABLE)

In [ ]:
VATGB = [value for value in VATs if value is not None and (value.startswith('GB'))]
options = webdriver.ChromeOptions()
options.headless = True

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(2)

wb1 = load_workbook(Chalfpointpath)
worksheet = wb1['Sheet1']
wait = WebDriverWait(driver, 20)
actions = ActionChains(driver)

print(VATGB)
for GBVATs in VATGB:
    if GBVATs is None:
        print("VAT not in SAP")
        continue

    driver.get(GBURL)
    GBverifybutton = '.govuk-button'
    GBVATinput = '#target'
    countryfromVAT = VATGB

    VATinput_element = driver.find_element(By.CSS_SELECTOR, GBVATinput)
    VATinput_element.send_keys(GBVATs)

    while retry_count <= max_retries:

        try:
            GBverifybutton_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, GBverifybutton)))
            GBverifybutton_element.click()
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            time.sleep(1.5)
            print("Button not found. Retrying...")
            retry_count += 1

    if retry_count < max_retries:
        print("Button clicked successfully")
    else:
        print("Button not found after maximum retries")

    time.sleep(2)

    GBpage_source = driver.page_source

    soup = BeautifulSoup(GBpage_source, 'html.parser')
    vat_number = soup.find("b", class_="bold").text.strip()

    business_name_heading = soup.find("h3", class_="govuk-heading-s")

    if business_name_heading:
        GBbusiness_name_element = business_name_heading.find_next("p", class_="govuk-body")
        GBbusiness_name = GBbusiness_name_element.text.strip() if GBbusiness_name_element else "Not found"
        GBvatstatus =  soup.find("h1", class_="govuk-panel__title").text.strip()
    else:
        GBbusiness_name = "Not found"
        GBvatstatus =  soup.find("h1", class_="govuk-heading-xl").text.strip()

    address_heading = soup.find("h3", text="Registered business address")

    if address_heading:
        address_element = address_heading.find_next_sibling("p", class_="govuk-body")

        if address_element:
            address_lines = [line.strip() for line in address_element.stripped_strings]
        else:
            address_lines = []
    else:
        address_lines = ["Not found"]

    GBaddress = ", ".join(address_lines)

    GBbody = soup.find("p", class_="govuk-body")

    GBnameandaddress = []

    if GBbusiness_name is not None and GBaddress is not None:
        lines = GBbody.get_text().strip().split('\n')
        br_tags = GBbody.find_all('br')
        alines = [line.strip() for line in GBbody.contents if line.strip()]

        name = lines[0].strip()

        GBnameandaddress.append(GBVATs)
        GBnameandaddress.append(GBbusiness_name)
        GBnameandaddress.append(GBaddress)
        GBnameandaddress.append(GBvatstatus)


        print("VAT Number:", GBVATs)
        print("Business Name:", GBbusiness_name)
        print("Business Address:", GBaddress)
        print(GBvatstatus)
        worksheet[f'A{row_counter}'] = GBVATs
        worksheet[f'B{row_counter}'] = GBbusiness_name
        worksheet[f'C{row_counter}'] = GBaddress
        worksheet[F'D{row_counter}'] = GBvatstatus

    else:
        print("Element with class 'govuk-body' not found in the HTML code")
        print(GBvatstatus)
        worksheet[f'A{row_counter}'] = GBVATs
        worksheet[F'D{row_counter}'] = GBvatstatus

    row_counter += 1

wb1.save(Cfinalpath)